In [ ]:
from bs4 import BeautifulSoup
import time
 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from tqdm.notebook import tqdm

from utils.send_email import send_email

In [ ]:
from utils.phd_keywords import phd_keywords

import os
from dotenv import load_dotenv

In [ ]:
def make_driver():
    options = Options()
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )

    return driver

In [ ]:
def login_to_linkedin(driver): 
    email =  os.environ["LINKEDIN_EMAIL_ADDRESS"]
    password =  os.environ["LINKEDIN_PASSWORD"]
    
    driver.get("https://linkedin.com/uas/login")
    time.sleep(5)
    username = driver.find_element("id", "username")
    username.send_keys(email)  
    pword = driver.find_element("id", "password")
    pword.send_keys(password)      
    driver.find_element("xpath", "//button[@type='submit']").click()    

In [ ]:
def find_positions(driver, phd_keywords):
    positions = []
    for keyword in tqdm(phd_keywords[:]):
        url = f'https://www.linkedin.com/search/results/content/?datePosted=%22past-24h%22&keywords="{keyword}"&origin=FACETED_SEARCH&sid=c%3Bi&sortBy=%22date_posted%22'
        driver.get(url)
        time.sleep(5)
        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(6)

        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        search_results = soup.find('main', {'aria-label': 'Search results'})

        positions += search_results.find_all('div', {'class': 'update-components-text relative feed-shared-update-v2__commentary'}) 
        
    return positions

In [ ]:
def clean_the_results(positions, position_keywords):
    results = []
    for position in positions:
        if any(item.lower() in position.text.lower() for item in position_keywords):
            results.append(position.text)    
    return results

In [ ]:
load_dotenv() 

In [ ]:
driver = make_driver()

In [ ]:
login_to_linkedin(driver)

In [ ]:
url = f'https://www.linkedin.com/search/results/content/?datePosted=%22past-24h%22&keywords="{phd_keywords[0]}"&origin=FACETED_SEARCH&sid=c%3Bi&sortBy=%22date_posted%22'
driver.get(url)

In [186]:
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

In [187]:
search_results = soup.find('main', {'aria-label': 'Search results'})

In [188]:
with open('search_results.txt', 'w') as f:
    f.write(str(search_results))

In [189]:
_positions = []

In [190]:
_positions += search_results.find_all('div', {'class': 'update-components-text relative feed-shared-update-v2__commentary'}) 


In [198]:
positions = [position.text.strip() for position in _positions]

In [199]:
urn_a = search_results.find_all('div', {'class': 'feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative feed-shared-update-v2--e2e artdeco-card'}) 

In [200]:
urn_b = search_results.find_all('div', {'class': 'feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative artdeco-card'}) 

In [201]:
len(urn_a)

32

In [205]:
position_b = []
for urn in urn_a + urn_b:
    try:
        position_text = urn.find('div', {'class': 'update-components-text relative feed-shared-update-v2__commentary'}).text.strip()
        if position_text:
            positions[positions.index(position_text)] = f'{position_text}\nhttps://www.linkedin.com/feed/update/{urn["data-urn"]}'
    except:
        pass

51

In [211]:
print(positions[-10])

Hi everyone,   I'm pleased to tell you about the following opportunity. Please share the information about this AHRC fully funded PhD in the form of a Collaborative Doctoral Award (CDA), now available via the Northern Bridge Consortium (NBC) Doctoral Training Partnership.  Mapping and Interpreting 90 years of Illustrating and Writing Childhood in the Collection of Seven Stories: The National Centre for Children's Books.  This PhD project will explore changing historical cultural constructions of childhood in books for children, making creative responses to these constructions, and looking at contemporary children's understanding of them. The context is the Seven Stories collection, a major, unique primary resource incorporating material from c.1930 to date, including original artwork and writing by key creators such as Edward Ardizzone, Judith Kerr, Yasmeen Ismail and Nick Sharratt. It will map and respond to key changes in the illustration and writing of childhood in children's books 

In [154]:
print(positions[3])

220 funded PhD positions in various departments at Ulster University UKDetail: https://lnkd.in/gxi3nsSbBachelor, Master, PhD, Postdoctoral Fellowship and academic and non academic opportunities at one place, join telegram for more opportunitieshttps://lnkd.in/eQ-v3iRD#research #students #opportunities #project #highereducation #university #science #phd #education #stem #innovation #email #cv #interview #universities #hiring #recuiting #career #masters #gra #graduaterecruitment #scholarship #graduate #research #students #opportunities #project #highereducation #university #science #postdoctoral #postdoc #postdocposition #postdoctoralfellowship #SVacademicconsultancy #Academic #consultancyservice #consultancy #academicresearch #academicwriter


In [161]:
len(set(positions+position_b))

95

In [ ]:
positions = find_positions(driver, phd_keywords[:])

In [ ]:
results = clean_the_results(positions, position_keywords)

In [ ]:
len(positions)

In [ ]:
    email_text = ""
    for result in set(results):
        email_text+=result.strip() + "\n" + "-"*50 + "\n"
        print(result.strip())
        print("-"*50, "\n")


In [ ]:
    if email_text != "":
        send_email("hue.salari@gmail.com", "Phd positions", email_text, "plain")